In [17]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from datetime import datetime

In [11]:
start_date = datetime(1760, 1, 1)
end_date = datetime(2024, 12, 31)

print(start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"))

17600101 20241231


In [38]:
import requests

url = f"https://codes.findlaw.com/LCsearch.html"
header= {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
      'Accept-Encoding': 'gzip, deflate, br',
      'Accept-Language': 'en-US,en;q=0.5',
      'Connection': 'keep-alive'}

response = requests.get(url, headers=header)
response

<Response [403]>

In [40]:
url = f"https://caselaw.findlaw.com/search.html?search_type=party&court=us-supreme-court&text=&date_start={start_date.strftime("%Y%m%d")}&date_end={end_date.strftime("%Y%m%d")}"
"https://caselaw.findlaw.com/search.html?search_type=party&court=us-supreme-court&text=&date_start=17600101&date_end=20241231"
header= {'User-Agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1"}

request = Request(url,None,header)
page = urlopen(request)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")


HTTPError: HTTP Error 403: Forbidden

In [62]:


options = webdriver.FirefoxOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
#options.add_argument("--headless")
driverService = Service('/snap/bin/geckodriver')
driver = webdriver.Firefox(service=driverService, options=options)

url = f"https://caselaw.findlaw.com/search.html?search_type=party&court=us-supreme-court&text=&date_start={start_date.strftime("%Y%m%d")}&date_end={end_date.strftime("%Y%m%d")}"

driver.get(url)
span = driver.find_element(By.CLASS_NAME, 'pagecount')
count = span.find_elements(By.TAG_NAME, 'strong')[1]
print(count.get_attribute("innerText"))

driver.close()

Error terminating service process.
Traceback (most recent call last):
  File "/home/kojo/miniconda3/envs/rag/lib/python3.12/site-packages/selenium/webdriver/common/service.py", line 170, in _terminate_process
    self.process.terminate()
  File "/home/kojo/miniconda3/envs/rag/lib/python3.12/subprocess.py", line 2209, in terminate
    self.send_signal(signal.SIGTERM)
  File "/home/kojo/miniconda3/envs/rag/lib/python3.12/subprocess.py", line 2201, in send_signal
    os.kill(self.pid, sig)
PermissionError: [Errno 13] Permission denied


1227


In [94]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By

class LawScraper:
    def __init__(self, headless: bool = True, driverPath : str = '/snap/bin/geckodriver') -> None:
        self.base_url = "https://caselaw.findlaw.com/search.html?search_type=party&court=us-supreme-court&text="
        options = webdriver.FirefoxOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        if headless: options.add_argument("--headless")
        driverService = Service(driverPath)
        self.driver = webdriver.Firefox(service=driverService, options=options)
        
    
    def get_cases(self, start: datetime, end: datetime, pageLimit: int = 20000) -> list[dict]:
        caseList = []
        self.driver.get(self.base_url + f"&date_start={start.strftime("%Y%m%d")}&date_end={end.strftime("%Y%m%d")}")
        span = self.driver.find_element(By.CLASS_NAME, 'pagecount')
        count = span.find_elements(By.TAG_NAME, 'strong')[1]
        pageCount = min(int(count.get_attribute("innerText")), pageLimit)
        print(pageCount)
        for p in range(1, pageCount+1):
            if p % 1000 == 1:
                print('On Page:', p)
            urls = self.get_page_urls(start, end, p)
            print(urls)
            for url in urls:
                caseList.append(self.get_case_content(url))
        return caseList

    def get_page_urls(self, start: datetime, end: datetime, pageNum : int) -> list[str]:
        self.driver.get(self.base_url + f"&date_start={start.strftime("%Y%m%d")}&date_end={end.strftime("%Y%m%d")}&page={pageNum}")
        table = self.driver.find_element(By.CLASS_NAME, 'responsive-card-table')
        linkTagList = table.find_elements(By.TAG_NAME, 'a')
        urlList = [a.get_attribute("href") for a in linkTagList]
        return urlList
    
    def get_case_content(self, url: str) -> dict:
        caseContent = {}
        self.driver.get(url)
        info = self.driver.find_element(By.CLASS_NAME, 'case-information')
        title = info.find_element(By.TAG_NAME, 'h2').get_attribute("innerText")
        caseContent['name'] = title.split(',')[0].split('(')[0].rstrip()

        paragraphs = info.find_elements(By.TAG_NAME, 'p')
        for paragraph in paragraphs:
            paragraphText = paragraph.get_attribute("innerText").split(':')
            caseContent[paragraphText[0]] = paragraphText[1]
            
        content = self.driver.find_element(By.ID, 'caselaw-content')
        caseContent['content'] = content.get_attribute("innerText")
        return caseContent
        

In [95]:
import json

scraper = LawScraper(headless=False)

start_date = datetime(1760, 1, 1)
end_date = datetime(2024, 12, 31)
caseDict = scraper.get_cases(start_date, end_date, 10)

with open("tempCases.json", 'w') as f:
    json.dump(caseDict, f)

10
On Page: 1
['https://caselaw.findlaw.com/court/us-supreme-court/187/436.html', 'https://caselaw.findlaw.com/court/us-supreme-court/396/275.html', 'https://caselaw.findlaw.com/court/us-supreme-court/430/723.html', 'https://caselaw.findlaw.com/court/us-supreme-court/355/22.html', 'https://caselaw.findlaw.com/court/us-supreme-court/375/398.html', 'https://caselaw.findlaw.com/court/us-supreme-court/368/349.html', 'https://caselaw.findlaw.com/court/us-supreme-court/419/1062.html', 'https://caselaw.findlaw.com/court/us-supreme-court/146/120.html', 'https://caselaw.findlaw.com/court/us-supreme-court/166/557.html', 'https://caselaw.findlaw.com/court/us-supreme-court/329/637.html', 'https://caselaw.findlaw.com/court/us-supreme-court/449/932.html', 'https://caselaw.findlaw.com/court/us-supreme-court/421/616.html', 'https://caselaw.findlaw.com/court/us-supreme-court/266/17.html', 'https://caselaw.findlaw.com/court/us-supreme-court/116/616.html', 'https://caselaw.findlaw.com/court/us-supreme-co

KeyboardInterrupt: 